In [3]:
using AxisArrays
using JuMP
using ProgressMeter
using Plots
gr()

Plots.GRBackend()

In [90]:
using MLDataPattern

In [93]:
using ScikitLearn
@sk_import svm: SVC

PyObject <class 'sklearn.svm.classes.SVC'>

In [126]:
ScikitLearn.predict(x::Float64, data) = x

In [133]:
include("mpc.jl")

MPC

In [195]:
Δt = 0.1

# m = 1.
# l = 1.
# g = 10.
# A = [0. 1.;
#     g/l 0.]
# B = [0. 1/(m*l^2.)]'
# sys = MPC.discretize(MPC.CTLinearSytstem(A, B), Δt)
sys = MPC.discretize(MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)

N = 20
time = Axis{:time}(linspace(0, (N - 1) * Δt, N))
side = Axis{:side}([:left, :right])
model = MPC.create_model(sys, time, side);

In [205]:
q0 = 2 * (rand() - 0.5)
v0 = 4 * (rand() - 0.5)
qlimb0 = q0 + 2 * (rand() - 0.5)
state = MPC.State(q0, v0, qlimb0)

result = MPC.solve!(model, state)
q = result.model.q
qlimb = result.model.qlimb
plt = plot(time.val, getvalue(q), label="CoM", xlabel="time (s)", ylabel="position", ylim=(-1.1, 1.1)) 
plot!(plt, time.val, getvalue(qlimb), label="foot")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 time (s) 
 
 
 position 
 
 
 
 
 
 
 
 CoM 
 
 
 
 foot

In [206]:
module s

include("mpc.jl")
using AxisArrays
using JuMP

immutable Example
    state::MPC.State{Float64}
    contact::AxisArray{Bool,2,Array{Bool,2},Tuple{Axis{:side,Array{Symbol,1}},Axis{:time,LinSpace{Float64}}}}
    objective::Float64
    vars::Dict{Symbol, Array{Float64}}
end

function sample()
    m = 1.
    l = 1.
    g = 10.
    A = [0. 1.;
        g/l 0.]
    B = [0. 1/(m*l^2.)]'
    lipm = MPC.discretize(MPC.CTLinearSytstem(A, B), 0.1)
    N = 20
    time = Axis{:time}(linspace(0, (N - 1) * lipm.Δt, N))
    side = Axis{:side}([:left, :right])
    model = MPC.create_model(lipm, time, side);
    q0 = 2 * (rand() - 0.5)
    v0 = 4 * (rand() - 0.5)
    qlimb0 = q0 + 2 * (rand() - 0.5)
    state = MPC.State(q0, v0, qlimb0)

    result = MPC.solve!(model, state)
    if result.status != :Optimal
        result = MPC.solve!(model, state, relax=true)
    end
    contact = map(x -> x > 0.5, getvalue(model.contact))
    obj = getvalue(getobjective(model.m))
    
    Example(state, contact, obj, Dict{Symbol, Array{Float64}}((k, getvalue.(v)) for (k, v) in model.m.varDict))
end

end

s

In [207]:
examples = @showprogress [s.sample() for i in 1:1000];

Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


In [208]:
ex_train, ex_test = splitobs(examples, 0.5);

In [209]:
function classify(side, time, examples)
    cls = SVC()
    data = hcat([[e.state.q, e.state.v, e.state.qlimb] for e in examples]...)
    labels = Float64.([e.contact[Axis{:side}(side), Axis{:time}(time)] for e in examples]);
    if length(unique(labels)) > 1
        fit!(cls, data', labels)
        return cls
    else
        return labels[1]
    end
end

classify (generic function with 1 method)

In [210]:
classifiers = AxisArray([classify(side, time, ex_train) for side in [:left, :right], time in 1:N], side, time)

2-dimensional AxisArray{Any,2,...} with axes:
    :side, Symbol[:left,:right]
    :time, linspace(0.0,1.9000000000000001,20)
And data, a 2×20 Array{Any,2}:
 PyObject SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)  …  0.0                                                                                                                                                                                                                                      
 PyObject SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)      PyObject SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf

In [218]:
ex = ex_test[end]

s.Example(s.MPC.State{Float64}(-0.5465523921228583,-1.8700843389781996,-0.762105135540931),Bool[true true … true false; false false … false true],851.948448094682,Dict{Symbol,Array{Float64,N}}(Pair{Symbol,Array{Float64,N}}(:contact,[1.0 1.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]),Pair{Symbol,Array{Float64,N}}(:q,[-0.546552,-0.663631,-0.674395,-0.628277,-0.555707,-0.474764,-0.395622,-0.323508,-0.26064,-0.207482,-0.163534,-0.127836,-0.0992559,-0.0766679,-0.0590426,-0.0454923,-0.0352899,-0.0281606,-0.0237381,-0.0215707]),Pair{Symbol,Array{Float64,N}}(:v,[-1.87008,-0.490969,0.273894,0.656148,0.807313,0.825021,0.770995,0.683284,0.584526,0.487492,0.398765,0.321145,0.255211,0.200305,0.155134,0.118126,0.0876182,0.0561547,0.0330322,0.0106747]),Pair{Symbol,Array{Float64,N}}(:qlimb,[-0.762105,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0]),Pair{Symbol,Array{Float64,N}}(:f,[19.9568 14.4024 … 0.00110956 -2.84217e-14; 1.42109e-14 1.42109e-14 … 0.0 -0.0])))

In [219]:
[predict(c, [ex.state.q ex.state.v ex.state.qlimb])[1] for c in classifiers]

2×20 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [220]:
ex.contact

2-dimensional AxisArray{Bool,2,...} with axes:
    :side, Symbol[:left,:right]
    :time, linspace(0.0,1.9000000000000001,20)
And data, a 2×20 Array{Bool,2}:
  true   true   true   true   true  …   true  false   true   true  false
 false  false  false  false  false     false  false  false  false   true

In [214]:
data = hcat([[e.state.q, e.state.v, e.state.qlimb] for e in examples]...)
labels = Float64.([e.contact[Axis{:side}(:right), Axis{:time}(5)] for e in examples]);

In [215]:
(x_train, y_train), (x_test, y_test) = splitobs((data, labels), at=0.5);

In [216]:

classifier = SVC()
fit!(classifier, x_train', y_train)
y_predict = predict(classifier, x_test');

In [217]:
mean(y_predict .== y_test)

0.96